<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_DIY04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
#@title
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext


# Faça você mesmo 4

1. Classe é a coluna class,portando Multiclass
2. Utilize MultiClassificationEvaluator
3. Use accuracy como métrica
4. Use um classificador diferente, por exemplo NaiveBayes.

> Importando as classes

In [99]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as f, types as t

In [100]:
schema = ('sepallength FLOAT, sepalwidth FLOAT, petallength FLOAT, petalwidth'
          ' FLOAT, class STRING')

dataset = spark.read.load(('/content/drive/MyDrive/Datasets/pyspark_course/iri'
          's.csv'),format='csv', header=True, schema=schema)

dataset.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

> Classes perfeitamente balanceadas.

In [101]:
dataset.select(f.col('class'))\
.groupBy(f.col('class'))\
.agg(f.count('class').alias('qtd')).show()

+---------------+---+
|          class|qtd|
+---------------+---+
| Iris-virginica| 50|
|    Iris-setosa| 50|
|Iris-versicolor| 50|
+---------------+---+



> Criando uma função para transformar a coluna CLASS em números inteiros.

In [102]:
def class_transformer(iris):
  try:
    classes = {
        'Iris-setosa': 0,
        'Iris-virginica': 1,
        'Iris-versicolor': 2
    }
    return classes[iris]
  except:
    return None

udf_class_trans = f.udf(lambda x: class_transformer(x), t.IntegerType())

> Aplicando em uma nova coluna.

In [103]:
dataset = dataset.withColumn('label', udf_class_trans('class'))

dataset.show()

+-----------+----------+-----------+----------+-----------+-----+
|sepallength|sepalwidth|petallength|petalwidth|      class|label|
+-----------+----------+-----------+----------+-----------+-----+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|    0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|    0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|    0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|    0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|    0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|    0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|    0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|    0|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|    0|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|    0|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|    0|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|    0|
|        4

> Vetorizando as variáveis independentes

In [104]:
assembler = VectorAssembler(inputCols=['sepallength', 'sepalwidth', 
            'petallength', 'petalwidth'], outputCol='vect_feats')

dataset = assembler.transform(dataset)

> Separando em treino e teste

In [105]:
dataTrain, dataTest = dataset.randomSplit([0.7, 0.3])

display(
    dataTrain.count(),
    dataTest.count()
)

104

46

> Escalonando os dados

In [106]:
scaler = StandardScaler(inputCol='vect_feats', outputCol='scaled_feats')
m_scaler = scaler.fit(dataTrain)

In [107]:
s_dataTrain = m_scaler.transform(dataTrain)
s_dataTrain.show()

+-----------+----------+-----------+----------+---------------+-----+--------------------+--------------------+
|sepallength|sepalwidth|petallength|petalwidth|          class|label|          vect_feats|        scaled_feats|
+-----------+----------+-----------+----------+---------------+-----+--------------------+--------------------+
|        4.3|       3.0|        1.1|       0.1|    Iris-setosa|    0|[4.30000019073486...|[5.28710831614822...|
|        4.4|       2.9|        1.4|       0.2|    Iris-setosa|    0|[4.40000009536743...|[5.41006420078657...|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|    0|[4.59999990463256...|[5.65597597006327...|
|        4.6|       3.2|        1.4|       0.2|    Iris-setosa|    0|[4.59999990463256...|[5.65597597006327...|
|        4.6|       3.4|        1.4|       0.3|    Iris-setosa|    0|[4.59999990463256...|[5.65597597006327...|
|        4.6|       3.6|        1.0|       0.2|    Iris-setosa|    0|[4.59999990463256...|[5.65597597006

> Instanciando, configurando e treinando o modelo

In [108]:
clf = NaiveBayes(featuresCol='scaled_feats', labelCol='label')
model = clf.fit(s_dataTrain)

> Escalonando os dados de teste

In [109]:
s_dataTest = m_scaler.transform(dataTest)
s_dataTest.show()

+-----------+----------+-----------+----------+--------------+-----+--------------------+--------------------+
|sepallength|sepalwidth|petallength|petalwidth|         class|label|          vect_feats|        scaled_feats|
+-----------+----------+-----------+----------+--------------+-----+--------------------+--------------------+
|        4.4|       3.0|        1.3|       0.2|   Iris-setosa|    0|[4.40000009536743...|[5.41006420078657...|
|        4.4|       3.2|        1.3|       0.2|   Iris-setosa|    0|[4.40000009536743...|[5.41006420078657...|
|        4.5|       2.3|        1.3|       0.3|   Iris-setosa|    0|[4.5,2.2999999523...|[5.53302008542492...|
|        4.7|       3.2|        1.6|       0.2|   Iris-setosa|    0|[4.69999980926513...|[5.77893185470162...|
|        4.8|       3.0|        1.4|       0.1|   Iris-setosa|    0|[4.80000019073486...|[5.90188832563988...|
|        4.8|       3.4|        1.9|       0.2|   Iris-setosa|    0|[4.80000019073486...|[5.90188832563988...|
|

> Prevendo usando os dados de teste

In [110]:
dataPredict = model.transform(s_dataTest)
dataPredict.select('class', 'label', 'prediction').show()

+--------------+-----+----------+
|         class|label|prediction|
+--------------+-----+----------+
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|Iris-virginica|    1|       1.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
|   Iris-setosa|    0|       0.0|
+--------------+-----+----------+
only showing top 20 rows



> Avaliando a performance do modelo

In [111]:
evl = MulticlassClassificationEvaluator(predictionCol='prediction',
                            labelCol='label', metricName='accuracy')

acc = evl.evaluate(dataPredict)

print(f'{round(acc * 100, 4)} % de Accuracy')

93.4783 % de Accuracy
